In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
import imageio
!pip install imageio-ffmpeg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 26.9 MB 48.0 MB/s 


In [21]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import pylab
from google.colab.patches import cv2_imshow
from moviepy.editor import VideoFileClip
#from moviepy import *
%matplotlib inline
import time
import os
import glob

RuntimeError: ignored

### Load weights and cfg

In [9]:
weights_path =os.path.join("/content/drive/MyDrive/project_data/yolov3.weights")
cfg_path =os.path.join("/content/drive/MyDrive/project_data/yolov3.cfg.txt")
print(" weights and cfg are loaded")

 weights and cfg are loaded


### load neural net in cv2 

In [10]:
net=cv2.dnn.readNetFromDarknet(cfg_path ,weights_path)

### Get layers name

In [11]:
names=net.getLayerNames()

In [13]:
layers_names=[names[i - 1] for i in net.getUnconnectedOutLayers()]

### define coco dataset labels

In [22]:
labels_path=os.path.join("/content/drive/MyDrive/project_data/coco.names.txt")
labels = open(labels_path).read().strip().split("\n")

### define the pipeline on each image 

In [23]:
def process(image):
  (H,W)=image.shape[:2]
  blob= cv2.dnn.blobFromImage(image,1/255.0,(416,416),crop=False,swapRB=False)
  net.setInput(blob)
  layers_output=net.forward(layers_names)
  boxes=[]
  confidences=[]
  classIDs=[]
  for output in layers_output:
    for detection in output:
        scores=detection[5:]
        classID=np.argmax(scores)
        confidence=scores[classID]
        
        if (confidence >0.85):
            box=detection[:4] * np.array([W,H,W,H])
            bx,by,bw,bh=box.astype("int")
            
            x=int(bx-(bw/2))
            y=int(by-(bh/2))
           
            boxes.append([x,y,int(bw),int(bh)])
            confidences.append(float(confidence))
            classIDs.append(classID)
    idx=cv2.dnn.NMSBoxes(boxes,confidences,0.8,0.8)
    if (len(idx)>0):
        for i in idx.flatten():
          (x,y)=[boxes[i][0],boxes[i][1]]
          (w,h)=[boxes[i][2],boxes[i][3]]
          cv2.rectangle (image,(x,y),(x+w ,y+h),(0,139,139),2)
          cv2.putText(image,"{}:{}:{}".format(labels[classIDs[i]],int(confidences[i]*100),"%"),(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,2.5,(0,139,139),10)
  return image

In [16]:
white_output = ("yolo_test1.mp4")
clip1 = VideoFileClip("/content/drive/MyDrive/project_data/project_video.mp4") 
white_clip = clip1.fl_image(process)
white_clip.write_videofile(white_output, audio=False)

NameError: ignored